In [ ]:
# === SETUP (run first) ===
!pip -q install ipywidgets

# Import potřebných knihoven
import numpy as np, cv2, matplotlib.pyplot as plt
from google.colab import files
import ipywidgets as W
from IPython.display import display

# Výpočet IoU mezi predikovanou maskou a GT maskou
def iou_score(pred_u8, gt_bool):
    pred = pred_u8 > 0                                        # binarizace predikce
    inter = np.logical_and(pred, gt_bool).sum()               # průnik
    union = np.logical_or(pred, gt_bool).sum()                # sjednocení
    return float(inter) / float(union) if union > 0 else 0.0  # výpočet IoU

# Převod vstupního obrazu na šedotónový uint8 formát
def to_gray_u8(img_bgr_or_gray):
    if img_bgr_or_gray.ndim == 3:                             # pokud je barevný
        g = cv2.cvtColor(img_bgr_or_gray, cv2.COLOR_BGR2GRAY)
    else:
        g = img_bgr_or_gray                                   # už je šedotónový
    if g.dtype != np.uint8:                                   # normalizace na rozsah 0–255
        g = cv2.normalize(g, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    return g

# Binarizace GT masky pomocí pevného prahu 128
def binarize_mask(mask_img):
    g = to_gray_u8(mask_img)
    _, m = cv2.threshold(g, 128, 255, cv2.THRESH_BINARY)
    return m > 0                                              # logická maska

# Hlavní vizualizační funkce – porovnání tří metod prahování
def tri_threshold_viz(img_bgr, gt_bool=None, user_start=None, title_prefix=""):
    gray = to_gray_u8(img_bgr)  # převod na šedotónový obraz

    # Výpočet prahů
    t_otsu, _ = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    t_otsu = int(t_otsu)
    t_fuzzy = int(gray.mean())                                # průměr intenzit jako fuzzy práh

    # Posuvník pro uživatelský práh
    slider = W.IntSlider(value=(user_start if user_start is not None else t_otsu),
                         min=0, max=255, step=1, description='User T')

    # Vnitřní funkce pro vykreslení výsledků
    def render(user_T):
        # Vytvoření binárních masek podle tří metod
        _, m_user  = cv2.threshold(gray, user_T, 255, cv2.THRESH_BINARY)
        _, m_otsu  = cv2.threshold(gray, t_otsu, 255, cv2.THRESH_BINARY)
        _, m_fuzzy = cv2.threshold(gray, t_fuzzy, 255, cv2.THRESH_BINARY)

        fig = plt.figure(figsize=(14,6))                      # nastavení velikosti výstupu

        # Originální obraz
        ax1 = plt.subplot(2,4,1)
        ax1.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
        ax1.set_title(f"{title_prefix}Originál"); ax1.axis('off')

        # GT maska (pokud je k dispozici)
        ax2 = plt.subplot(2,4,2)
        if gt_bool is not None:
            ax2.imshow(gt_bool, cmap='gray'); ax2.set_title("GT maska")
        else:
            ax2.text(0.05, 0.6, "GT maska: N/A", fontsize=12); ax2.set_title("Info")
        ax2.axis('off')

        # Histogram s vyznačenými prahy
        ax3 = plt.subplot(2,4,3)
        ax3.hist(gray.ravel(), bins=256, range=(0,256))
        ax3.axvline(user_T, color='C1', linestyle='--', label=f'User {user_T}')
        ax3.axvline(t_otsu, color='C2', linestyle='--', label=f'Otsu {t_otsu}')
        ax3.axvline(t_fuzzy, color='C3', linestyle='--', label=f'Fuzzy {t_fuzzy}')
        ax3.set_title("Histogram s prahy"); ax3.legend()

        # IoU graf (pokud je GT maska)
        ax4 = plt.subplot(2,4,4)
        if gt_bool is not None:
            vals = [iou_score(m_user, gt_bool), iou_score(m_otsu, gt_bool), iou_score(m_fuzzy, gt_bool)]
            ax4.bar(["User","Otsu","Fuzzy"], vals); ax4.set_ylim(0,1); ax4.set_title("IoU vs. GT")
            ax4.grid(True, axis='y', alpha=0.3)
        else:
            ax4.axis('off'); ax4.text(0.0, 0.6, "IoU: bez GT nepočítáme", fontsize=12)

        # Výsledné binární masky
        ax5 = plt.subplot(2,4,5); ax5.imshow(m_user,  cmap='gray'); ax5.set_title("User");  ax5.axis('off')
        ax6 = plt.subplot(2,4,6); ax6.imshow(m_otsu,  cmap='gray'); ax6.set_title("Otsu");  ax6.axis('off')
        ax7 = plt.subplot(2,4,7); ax7.imshow(m_fuzzy, cmap='gray'); ax7.set_title("Fuzzy"); ax7.axis('off')

        # Textové shrnutí prahových hodnot
        ax8 = plt.subplot(2,4,8); ax8.axis('off')
        ax8.text(0.0, 0.8, f"User T:  {user_T}", fontsize=12)
        ax8.text(0.0, 0.6, f"Otsu T:  {t_otsu}", fontsize=12)
        ax8.text(0.0, 0.4, f"Fuzzy T: {t_fuzzy}", fontsize=12)

        plt.tight_layout(); plt.show()

    # Aktivace interaktivního posuvníku
    W.interact(render, user_T=slider)


In [ ]:
# === MODE A: Synthetic + GT ===
# Nastavení základních parametrů syntetického obrazu (tvar, kontrast, šum)
shape = "circle"     # Tvar objektu: 'circle' nebo 'rect'
h, w = 256, 256      # Rozměry obrazu (výška, šířka)
fg = 190             # Intenzita objektu (foreground) – světlý
bg = 30              # Intenzita pozadí (background) – tmavý
radius = 60          # Poloměr kruhu nebo poloviční šířka obdélníku
noise_sigma = 10     # Standardní odchylka šumu (0–30)

# Inicializace generátoru náhodných čísel pro konzistentní výstup
rng = np.random.default_rng(123)

# Vytvořeí obrazu vyplněný pozadím
img = np.full((h, w), bg, dtype=np.float32)

# Inicializace GT masky jako nulovou
gt = np.zeros((h, w), dtype=np.uint8)

# Určení středu obrazu
cy, cx = h//2, w//2

# Vytvoření binární masky objektu podle zvoleného tvaru
if shape == "circle":
    Y, X = np.ogrid[:h, :w]
    mask = (X - cx)**2 + (Y - cy)**2 <= radius**2  # kruhová maska
else:
    # obdélníková maska s výškou cca 70 % šířky
    mask = (np.abs(np.arange(w)-cx)[None,:] < radius) & \
           (np.abs(np.arange(h)-cy)[:,None] < int(radius*0.7))

# Nastavení intenzity objektu v obraze
img[mask] = fg

# Gaussovský šum
img += rng.normal(0, noise_sigma, img.shape)

# Oříznutí hodnot do rozsahu 0–255 a převedení na uint8
img = np.clip(img, 0, 255).astype(np.uint8)

# GT maska: 255 pro objekt, 0 pro pozadí
gt[mask] = 255
gt_bool = gt > 0

# Vizualizace tří metod prahování na syntetickém obraze
tri_threshold_viz(
    cv2.cvtColor(img, cv2.COLOR_GRAY2BGR),  # převod na BGR pro zobrazení
    gt_bool=gt_bool,                        # GT maska pro evaluaci
    user_start=None,                        # výchozí uživatelský práh (None = použije Otsu)
    title_prefix="Syntetika | "             # prefix pro titulky grafů
)


In [ ]:
# === MODE B: Upload image + GT mask ===

print("Nahraj 2 soubory: 1) obraz (PNG/JPG/...), 2) GT masku (binární 0/255).")

up = files.upload()

# Kontrola, zda byly nahrány alespoň dva soubory
if len(up) < 2:
    raise SystemExit("Nutné nahrát obraz + GT masku.")

img_name, mask_name = list(up.keys())[:2]

# Načtení obrazu z binárního bufferu jako barevný (BGR)
img_bgr = cv2.imdecode(np.frombuffer(up[img_name], np.uint8), cv2.IMREAD_COLOR)
if img_bgr is None:
    raise SystemExit("Obraz se nepodařilo načíst.")

# Načtení masky z binárního bufferu
mask_any = cv2.imdecode(np.frombuffer(up[mask_name], np.uint8), cv2.IMREAD_UNCHANGED)
if mask_any is None:
    raise SystemExit("Maska se nepodařila načíst.")

# Zkontroluje rozměry a případně dorovná masku na velikost obrazu pomocí nearest-neighbor interpolace
gh, gw = img_bgr.shape[:2]
if mask_any.shape[:2] != (gh, gw):
    mask_any = cv2.resize(mask_any, (gw, gh), interpolation=cv2.INTER_NEAREST)

# Převee masku na binární logickou mapu (True = popředí)
gt_bool = binarize_mask(mask_any)

# Vizualizaca tří metod prahování na nahraném obraze s GT maskou
tri_threshold_viz(
    img_bgr,                        # vstupní obraz
    gt_bool=gt_bool,                # GT maska pro výpočet IoU
    user_start=None,                # výchozí uživatelský práh (None = použije Otsu)
    title_prefix=f"{img_name} | "   # prefix pro titulky grafů
)


In [ ]:
# === MODE C: Upload image only (no GT) ===

print("Vyber 1 soubor s obrazem (PNG/JPG/BMP/TIFF).")

up = files.upload()

# Kontrola, zda byl skutečně nahrán alespoň jeden soubor
if not up:
    raise SystemExit("Nebyl nahrán žádný soubor.")

img_name = next(iter(up))

# Načtení obrazu z binárního bufferu jako barevný (BGR)
img_bgr = cv2.imdecode(np.frombuffer(up[img_name], np.uint8), cv2.IMREAD_COLOR)
if img_bgr is None:
    raise SystemExit("Soubor se nepodařilo načíst jako obrázek.")

# Vizualizace tří metod prahování bez GT masky
tri_threshold_viz(
    img_bgr,                        # vstupní obraz
    gt_bool=None,                   # GT maska není k dispozici
    user_start=None,                # výchozí uživatelský práh (None = použije Otsu)
    title_prefix=f"{img_name} | "   # prefix pro titulky grafů
)
